In [1]:
import numpy as np
import tnn
import torch
import torch.nn as nn
import torch.utils.data as data
import logging

from torch.optim.lr_scheduler import ReduceLROnPlateau
from datasets import load_dataset, concatenate_datasets
from transformers import BertTokenizer

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

In [2]:
dataset = load_dataset("cardiffnlp/tweet_eval", "emoji")
train = dataset.get("train")
val = dataset.get("validation")

In [3]:
classes = len(train.unique("label"))
name = "distilbert/distilbert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(name)
pad_id = tokenizer.get_vocab().get("[PAD]")

/home/andrew/git/teaching-neural-networks/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
length =0
for example in train:
    length = max(length, len(tokenizer(example["text"]).input_ids))

for example in val:
    length = max(length, len(tokenizer(example["text"]).input_ids))

print(f"longest sequence: {length}")

longest sequence: 75


In [5]:
def pre_process(examples):
    examples = tokenizer(
        examples["text"],
        max_length=96,
        truncation=True,
        padding="max_length",
    )
    return examples


train_dataset = train.map(
    pre_process, batched=True, num_proc=2, remove_columns=["text"]
)
val_dataset = val.map(
    pre_process, batched=True, num_proc=2, remove_columns=["text"]
)

In [6]:
def collate_fn(batch):
    inputs = {"input_ids": [], "token_type_ids": [], "attention_mask": []}
    labels = []

    for example in batch:
        inputs["input_ids"].append(example["input_ids"])
        inputs["token_type_ids"].append(example["token_type_ids"])
        inputs["attention_mask"].append(example["attention_mask"])
        labels.append(example["label"])

    inputs["input_ids"] = torch.tensor(inputs["input_ids"]).long()
    inputs["token_type_ids"] = torch.tensor(inputs["token_type_ids"]).long()
    inputs["attention_mask"] = torch.tensor(inputs["attention_mask"]).long()
    labels = torch.tensor(labels).long()
    return inputs, labels

In [7]:
trainloader = data.DataLoader(
    train_dataset,
    batch_size=64,
    collate_fn=collate_fn,
    drop_last=False,
    shuffle=False,
    pin_memory=True,
    num_workers=2,
)

testloader = data.DataLoader(
    val_dataset,
    batch_size=128,
    collate_fn=collate_fn,
    drop_last=False,
    shuffle=False,
    pin_memory=True,
    num_workers=2,
)


def to(inputs, labels, device, non_blocking):
    inputs = dict(
        map(
            lambda item: (item[0], item[1].to(device, non_blocking=non_blocking)),
            inputs.items(),
        )
    )
    labels = labels.to(device, non_blocking=True)
    return inputs, labels

In [8]:
model = tnn.BertForClassification(classes=classes, hidden_size=768, name=name)
lr = 2e-4
weight_decay = 1e-2
factor = 0.1
patience = 5

optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
loss_fn = nn.CrossEntropyLoss(ignore_index=pad_id)
scheduler = ReduceLROnPlateau(optim, mode="min", factor=factor, patience=patience)

In [9]:
trainer = tnn.Trainer(
    model,
    optim,
    loss_fn,
    trainloader,
    testloader,
    scheduler=scheduler,
    epochs=100,
    unpack_inputs=True,
    save_weights=True,
    device=device,
    to_fn=to,
    path="../training/emotions-bert-sgdm.h5",
    verbose=True,
    profile=False,
)

In [ ]:
sgdm_metrics = trainer.train()

model using cuda
weights saved to ../training/emotions-bert-sgdm.h5/trajectory/weights-epoch-0
training started


In [ ]:
model = tnn.BertForClassification(classes=classes, hidden_size=768, name=name)
optim = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, nesterov=True, weight_decay=weight_decay)
scheduler = ReduceLROnPlateau(optim, mode="min", factor=factor, patience=patience)

In [ ]:
trainer.model = model
trainer.optim = optim
trainer.scheduler = scheduler
trainer.path = "../training/emotions-bert-sgdm-nesterov.h5"
sgdm_nesterov_metrics = trainer.train()

In [ ]:
model = tnn.BertForClassification(classes=classes, hidden_size=768, name=name)
optim = torch.optim.RMSprop(model.parameters(), lr=lr, alpha=0.99, weight_decay=weight_decay)
scheduler = ReduceLROnPlateau(optim, mode="min", factor=factor, patience=patience)

In [ ]:
trainer.model = model
trainer.optim = optim
trainer.scheduler = scheduler
trainer.path = "../training/emotions-bert-rmsprop.h5"
rmsprop_metrics = trainer.train()

In [ ]:
model = tnn.BertForClassification(classes=classes, hidden_size=768, name=name)
optim = torch.optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999), weight_decay=weight_decay)
scheduler = ReduceLROnPlateau(optim, mode="min", factor=factor, patience=patience))

In [ ]:
trainer.model = model
trainer.optim = optim
trainer.scheduler = scheduler
trainer.path = "../training/emotions-bert-adam.h5"
adam_metrics = trainer.train()